In [24]:
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator
import matplotlib.pyplot as plt
import numpy as np

In [37]:
IMAGE_SIZE = 224
train_datagen = ImageDataGenerator(
        rotation_range=40,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)
test_datagen = ImageDataGenerator(rescale=1./255)

train_generator = train_datagen.flow_from_directory(
        'seefood/train',
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=32,
        class_mode='binary')

test_generator = test_datagen.flow_from_directory(
        'seefood/test',
        target_size=(IMAGE_SIZE, IMAGE_SIZE),
        batch_size=64,
        class_mode='binary')



Found 498 images belonging to 2 classes.
Found 500 images belonging to 2 classes.


In [26]:
base_model = tf.keras.applications.MobileNetV2(input_shape=(IMAGE_SIZE, IMAGE_SIZE, 3),
                                               include_top=False,
                                               weights='imagenet')
base_model.trainable = False      #freeze the base model. Pre-trained 

In [27]:
#We have 7x7 activation maps rn. GlobalAveragePooling will give us 7x7 pooled 1280 size layer. (Instead of flattening)
global_average_layer = tf.keras.layers.GlobalAveragePooling2D()

In [28]:
#only two classes. so a dense layer with one neuron
#You don't need an activation function here because this prediction will be treated as a logit, or a raw prediction value. 
#Positive numbers predict class 1, negative numbers predict class 0.
prediction_layer = tf.keras.layers.Dense(1)

In [29]:
model = tf.keras.Sequential([
  base_model,
  global_average_layer,
  prediction_layer
])

In [39]:
base_learning_rate = 0.01    #very small to ensure no major changes

#Since there are two classes, use a binary cross-entropy loss with from_logits=True since the model provides a linear output.
model.compile(#optimizer=tf.keras.optimizers.RMSprop(lr=base_learning_rate),
              optimizer='adam'
              loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
              metrics=['accuracy'])

SyntaxError: invalid syntax (<ipython-input-39-38a1e103b57f>, line 6)

In [40]:
model.fit(train_generator, steps_per_epoch=8, epochs=10)

  ...
    to  
  ['...']
Train for 8 steps
Epoch 1/10
8/8 [==============================] - 23s 3s/step - loss: 1.5271 - accuracy: 0.6250
Epoch 2/10
8/8 [==============================] - 21s 3s/step - loss: 0.9228 - accuracy: 0.6914
Epoch 3/10
8/8 [==============================] - 20s 3s/step - loss: 0.5389 - accuracy: 0.7975
Epoch 4/10
8/8 [==============================] - 25s 3s/step - loss: 0.6643 - accuracy: 0.7645
Epoch 5/10
8/8 [==============================] - 32s 4s/step - loss: 0.4416 - accuracy: 0.7975
Epoch 6/10
8/8 [==============================] - 24s 3s/step - loss: 0.4164 - accuracy: 0.8164
Epoch 7/10
8/8 [==============================] - 21s 3s/step - loss: 0.6824 - accuracy: 0.7521
Epoch 8/10
8/8 [==============================] - 32s 4s/step - loss: 0.3937 - accuracy: 0.8512
Epoch 9/10
8/8 [==============================] - 25s 3s/step - loss: 0.8272 - accuracy: 0.7066
Epoch 10/10
8/8 [==============================] - 20s 3s/step - loss: 0.6717 - accuracy: 0.7

In [41]:
model.evaluate(test_generator)

  ...
    to  
  ['...']
8/8 [==============================] - 11s 1s/step - loss: 0.3622 - accuracy: 0.8740


[0.3622194118797779, 0.874]

In [42]:
model.save('trained_hotdog.h5')